In [ ]:
!pip install -q -U langchain langchain-community langchain-core langchain-text-splitters langchain-google-genai faiss-cpu langgraph langchain-huggingface

In [ ]:
import os
import getpass
import torch
from google.colab import userdata

#api key setup
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

repo_url = "https://github.com/progyan01/DSA-Coach.git"
repo_name = "DSA-Coach"

if not os.path.exists(repo_name):
    print(f"Cloning repository: {repo_url}")
    !git clone {repo_url}
else:
    print("Repository already exists. Pulling latest changes...")
    !cd {repo_name} && git pull

print("Loading notes...")
loader = DirectoryLoader(f"./{repo_name}/data", glob="**/*.md", loader_cls=TextLoader)
docs = loader.load()
print(f"Loaded {len(docs)} documents.")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=200,
    separators=["\n# ", "\n## ", "\n### ", "```", "\n$$", "\n\n", "\n", " "]
)
splits = text_splitter.split_documents(docs)
print(f"Created {len(splits)} chunks for the vector store.")

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
    query_instruction="Represent this query for retrieving relevant coding and algorithm documentation: "
)

vectorstore = FAISS.from_documents(splits, embeddings)

print("Success!")

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

retriever = vectorstore.as_retriever()
tool = create_retriever_tool(
    retriever,
    "search_dsa_notes",
    "Searches the notes for algorithms, data structures, and code examples."
)
tools = [tool]

system_prompt = """
You are "DSA-Coach", an expert Technical Interviewer.
Your goal is to help students solve problems by guiding them, NOT by giving the answer.

### PHASE 1: ANALYZE THE INPUT
The user input might contain:
1. A Story/Problem Statement (e.g., "Alice needs to visit cities...").
2. Code (optional).
3. An Error Message (optional).

**CRITICAL STEP (Story Translation):**
If the user provides a "Story Problem" without naming an algorithm:
1. You must DECONSTRUCT the story.
2. Identify the underlying CS Concept (e.g., "Cities are Nodes" -> "Graph").
3. Identify the Algorithm (e.g., "Shortest path with weights" -> "Dijkstra").
4. ONLY THEN use the tool to search for that specific algorithm.

### PHASE 2: DEBUGGING PROTOCOL
- **Syntax Errors:** If the code has simple syntax errors (missing semicolon, indentation), point them out immediately. DO NOT use tools for this.
- **Logic Errors (Silent Failures):** If the code runs but is wrong (or TLE), compare the user's logic to the Correct Algorithm found in the notes.
- **Guardrail:** DO NOT write the full corrected code. Give a hint: "You missed the base case for recursion."
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

agent_executor = create_react_agent(
    llm,
    tools,
    prompt=prompt
)

print("Agent is ready to use!")

In [ ]:
from IPython.display import display, Markdown

def ask_coach(code=None, error=None, question=""):
    print("(Type 'exit' to quit)\n")
    while(True):
      question = input("Enter Question: ")
      if question.lower() == 'exit': break
      code = input("Paste Code (press Enter if none): ")
      error = input("Paste Error (press Enter if none): ")

      user_context = f"QUESTION: {question}\n\n"
      if code:
          user_context += f"STUDENT CODE:\n{code}\n\n"
      if error:
          user_context += f"ERROR/ISSUE:\n{error}"
      elif code:
          user_context += "ISSUE: Code runs but produces wrong output (Silent Failure)."

      print("DSA-Coach is thinking...")
      try:
          result = agent_executor.invoke({
              "messages": [HumanMessage(content=user_context)]
          })

          answer = result["messages"][-1].content
          final_answer=answer[0]["text"]
          display(Markdown(f"###Coach's Feedback\n{final_answer}"))

      except Exception as e:
          print(f"❌ Error: {e}")
